## 5.6　Affine／Softmax レイヤの実装

In [1]:
import numpy as np

### 5.6.1　Affine レイヤ

ニューラルネットワークの順伝播で行う行列の積は、幾何学の分野では「**アフィン変換**」と呼ばれている。

In [2]:
X = np.random.rand(2)
W = np.random.rand(2, 3)
B = np.random.rand(3)
Y = np.dot(X, W) + B

print(X.shape)
print(W.shape)
print(B.shape)

(2,)
(2, 3)
(3,)


$ (0 \times 2) \times (2 \times 3) $ で内側が揃っているので、外側の $ (0 \times 3) $ が計算結果となる。

### 5.6.2　バッチ版 Affine レイヤ

In [3]:
X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])

print(X_dot_W)
print((X_dot_W + B))

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


In [4]:
dY = np.array([[1, 2, 3], [4, 5, 6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [5]:
dB = np.sum(dY, axis=0)
dB

array([5, 7, 9])

### 5.6.3　Softmax-with-Loss レイヤ

ニューラルネットの学習時には Softmax レイヤは必要になるが、推論のときには必要でない。<br>
また、ニューラルネットワークの Softmax レイヤの逆伝播は、出力と教師ラベルの誤差を前レイヤに伝えていく。

In [6]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax()
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

In [7]:
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size